In [775]:
import os
import sys
import requests
import pandas as pd
import numpy as np
import re
from dotenv import load_dotenv
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.embeddings.base import Embeddings
from sklearn.metrics.pairwise import cosine_similarity

In [776]:
# Load environment variables
load_dotenv()

True

In [777]:

API_KEY = os.getenv("IBM_WATSONX_API_KEY")
PROJECT_ID = os.getenv("IBM_WATSONX_PROJECT_ID")
API_URL = os.getenv("IBM_WATSONX_URL")
MODEL_ID = "sdaia/allam-1-13b-instruct"

In [778]:
# Initialize the embedding function
emb_func = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

In [779]:
def get_access_token():
    token_url = "https://iam.cloud.ibm.com/identity/token"
    headers = {"Content-Type": "application/x-www-form-urlencoded"}
    data = {
        "grant_type": "urn:ibm:params:oauth:grant-type:apikey",
        "apikey": API_KEY
    }
    response = requests.post(token_url, headers=headers, data=data)
    if response.status_code == 200:
        return response.json()["access_token"]
    else:
        raise Exception(f"Error retrieving token: {response.text}")


In [780]:

def load_and_clean_data(file_path):
    df = pd.read_csv(file_path, nrows=1000)
    df = df.astype(str).dropna()
    df['text'] = df[['العصر', 'الشاعر', 'الديوان', 'القافية', 'البحر', 'الشطر الايسر', 'الشطر الايمن', 'البيت']].agg(' '.join, axis=1)
    return df

In [781]:
def create_embeddings(df):
    embeddings_file = 'poem_embeddings.npy'
    if os.path.exists(embeddings_file):
        print("Loading existing embeddings...")
        return np.load(embeddings_file)
    else:
        print("Creating new embeddings...")
        embeddings = emb_func.embed_documents(df['text'].tolist())
        np.save(embeddings_file, embeddings)
        return np.array(embeddings)


In [782]:
def normalize_letter(letter):
    return 'ا' if letter in ['أ', 'إ', 'ء', 'ى'] else letter

In [783]:
def starts_with_letter(verse, letter):
    if letter is None:
        return True  # Allow any letter if last_letter is None
    verse = re.sub(r'[\u064B-\u0652]', '', verse)
    letter = re.sub(r'[\u064B-\u0652]', '', letter)
    return normalize_letter(verse[0]) == normalize_letter(letter)

In [784]:
def retrieve_similar_verses(query, last_letter, embeddings, df, top_k=10):
    query_embedding = emb_func.embed_query(query)
    similarities = cosine_similarity([query_embedding], embeddings)[0]
    
    mask = df['البيت'].apply(lambda x: starts_with_letter(x, last_letter))
    filtered_similarities = similarities * mask
    
    top_indices = np.argsort(filtered_similarities)[-top_k:][::-1]
    return df.iloc[top_indices]

In [785]:
def get_last_letter(verse):
    # Remove diacritics and non-word characters
    verse = re.sub(r'[ًٌٍَُِّْـ\W]', '', verse)
    # List of possible elongations
    elongations = ['ا', 'و', 'ي', 'ها', 'ما', 'با', 'سا', 'دا', 'غا', 'فا', 'طا', 'جا', 'زا', 'شا', 'عا', 'قا', 'لا', 'نا', 'وا']
    # List of possible endings to ignore
    endings_to_ignore = ['ة', 'ه', 'هم', 'هن', 'هما']
    
    if not verse:
        return None
    
    # Check if the verse ends with an elongation
    for elongation in elongations:
        if verse.endswith(elongation):
            return verse[-len(elongation)-1] if len(verse) > len(elongation) else verse[0]
    
    # Check if the verse ends with any of the endings to ignore
    for ending in endings_to_ignore:
        if verse.endswith(ending):
            return verse[-len(ending)-1]
    
    return verse[-1]

In [786]:
def generate_response(prompt, access_token):
    url = "https://eu-de.ml.cloud.ibm.com/ml/v1/text/generation?version=2023-05-29"
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {access_token}",
    }
    data = {
        "input": prompt,
        "parameters": {
		"decoding_method": "greedy",
		"max_new_tokens": 600,
		"min_new_tokens": 0,
		"stop_sequences": [],
		"repetition_penalty": 1
	},
        "model_id": MODEL_ID,
        "project_id": PROJECT_ID
    }
    response = requests.post(url, headers=headers, json=data)
    if response.status_code == 200:
        return response.json()['results'][0]['generated_text']
    else:
        print(f"Error generating response from Allam model: {response.text}")
        return None

In [787]:
def play_jazil():
    df = load_and_clean_data('Arabic Poem Comprehensive Dataset (APCD).csv')
    embeddings = create_embeddings(df)
    access_token = get_access_token()
    
    print("Welcome to Jazil! Let's start the poetic exchange.")
    print("يمكنك البدء ببيت شعري عشوائي.")
    
    last_letter = None
    ai_turn = False
    used_verses = set()
    conversation_history = []

    while True:
        if not ai_turn:
            user_input = input("دورك (أو اكتب 'quit' للخروج): ")
            if user_input.lower() == 'quit':
                print("شكرًا للعب في جزيل!")
                break
            
            if user_input in used_verses:
                print("هذا البيت قد تم استخدامه بالفعل. يرجى تقديم بيت مختلف.")
                continue

            if last_letter and not starts_with_letter(user_input, last_letter):
                print(f"يجب أن يبدأ بيتك بالحرف '{last_letter}'. حاول مرة أخرى.")
                continue

            # Verify the verse using RAG
            similar_verses = retrieve_similar_verses(user_input, last_letter, embeddings, df)
            context = "\n".join(similar_verses['البيت'].tolist())
            
            prompt = f"""
            أنت خبير متخصص في الشعر العربي الكلاسيكي. مهمتك هي تحليل النص المقدم وتحديد ما إذا كان بيتًا شعريًا صحيحًا أم لا.
            قم بتقييم النص التالي بدقة شديدة وفقًا للمعايير الآتية:
            1. الشكل: هل يتكون النص من صدر وعجز متوازنين؟
            2. الوزن الشعري: هل يتبع النص أحد البحور الشعرية العربية المعروفة؟
            3. القافية: هل توجد قافية واضحة ومناسبة في نهاية البيت؟
            4. اللغة: هل يستخدم النص لغة عربية فصحى وتراكيب شعرية تقليدية؟
            5. المعنى: هل للنص معنى شعري واضح ومتماسك؟
            6. الأصالة: هل يبدو النص كأنه من الشعر العربي الكلاسيكي وليس مجرد جملة عادية أو نصًا حديثًا؟

            النص المراد تقييمه:
            {user_input}

            أمثلة على أبيات شعرية مشابهة:
            {context}

            بعد التحليل الدقيق، قدم إجابة مفصلة:
            1. صنف النص إما كـ "بيت شعري صحيح" أو "ليس بيتًا شعريًا صحيحًا".
            2. اشرح باختصار سبب تصنيفك، مع الإشارة إلى المعايير التي تم استيفاؤها أو انتهاكها.
            """
            verification_response = generate_response(prompt, access_token)

            if "بيت شعري صحيح" in verification_response.lower():
                print(f"المستخدم: {user_input}")
                used_verses.add(user_input)
                last_letter = get_last_letter(user_input)
                conversation_history.append(f"User: {user_input}")
                ai_turn = True
            else:
                print("هذا النص ليس بيتًا شعريًا صحيحًا. يرجى تقديم بيت شعري حقيقي.")
                print("سبب الرفض:", verification_response)
                continue

        else:
            print("دور الذكاء الاصطناعي...")
            
            similar_verses = retrieve_similar_verses(" ".join(conversation_history[-3:]), last_letter, embeddings, df)
            context = "\n".join(similar_verses['البيت'].tolist())
            
            prompt = f"""
            أنت شاعر عربي ماهر. مهمتك هي إنتاج بيت شعري واحد فقط يتناسب مع سياق المحادثة الحالية ويبدأ بالحرف المطلوب.

            سياق المحادثة:
            {' '.join(conversation_history[-3:])}

            الحرف الذي يجب أن يبدأ به البيت الجديد: {last_letter if last_letter else 'أي حرف'}

            أمثلة على أبيات شعرية مشابهة:
            {context}
            خذ بيت شعري واحد فقط من هذه الأمثلة.
            قم بإنتاج بيت شعري جديد يتبع القواعد التالية:
            1. يبدأ بالحرف المحدد (إذا كان محددًا).
            2. يتكون من صدر وعجز متوازنين، مفصولين بسطر جديد.
            3. يتبع أحد البحور الشعرية العربية المعروفة.
            4. يحتوي على قافية مناسبة.
            5. يستخدم لغة عربية فصحى وتراكيب شعرية تقليدية.
            6. له معنى شعري واضح ومتماسك.
            7. يبدو كجزء أصيل من الشعر العربي الكلاسيكي.

            ملاحظة مهمة: أنتج بيت شعري واحد فقط، لا تضف أي شرح أو تعليق. اكتب الصدر في سطر والعجز في السطر التالي.

            أنتج البيت الشعري:
            """
            generated_text = generate_response(prompt, access_token)
            # generated_verse = extract_verse(generated_text)

            if generated_text and generated_text not in used_verses:
                print(f"الذكاء الاصطناعي: {generated_text}")
                used_verses.add(generated_text)
                last_letter = get_last_letter(generated_text)  # Get last letter from the second line (عجز)
                conversation_history.append(f"AI: {generated_text}")
                ai_turn = False
            else:
                print("لم أستطع إنتاج بيت شعري مناسب. دورك.")
                ai_turn = False

# Run the game
play_jazil()

Loading existing embeddings...
Welcome to Jazil! Let's start the poetic exchange.
يمكنك البدء ببيت شعري عشوائي.
المستخدم: دع المساجد للــعباد تسكنها ‏وطف بنا حول خمار ليسقينا
دور الذكاء الاصطناعي...
الذكاء الاصطناعي: 
            نَبَتَ العِشقُ في الفُؤادِ كَما يَنتَشي    نَبضُ القَلبِ في الجَوانِحِ وَالأَضلُعِ 
المستخدم: عفا الله عن ليلى الغداة فإنها ** إذا وليت حكما علي تجور
دور الذكاء الاصطناعي...
الذكاء الاصطناعي: صَدر: رَأى البَحرَ أَبدى لَه سَطوَرَ مُزبرَجا
            عجز: فَقالَ وَراءَكَ ما تَزَعزَعُ أَسرُجُهُ 
المستخدم: جرينا على أهدى طريق ومنهاج ** بنورِ سراجِ للنبوءة وهَاج
دور الذكاء الاصطناعي...
الذكاء الاصطناعي: صدر: جِنانُ الخُلدِ أَرضٌ جَمَّةُ الثَمنِ
            عجز: جَنى الجَاني عَلَيها ما جَنى الثَمنِ 
المستخدم: نَسيتُ وَما أَنسى عِتاباً عَلى الصِدِّ وَلا خَفَراً زادَت بِهِ حُمرَةُ الخَدِّ  
دور الذكاء الاصطناعي...
الذكاء الاصطناعي: صدر: دَمعٌ جَرى فَقَضى في الرَبعِ ماوَجَبا
            عجز: لِأَهلِهِ وَشَفى أَنّى وَلا كَرَبا 
يجب أن يبدأ بيتك بالحرف 'ب'. حاول مرة أخر